# Dataset inspection

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last"
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

## Analysing and removing careless or biased participations

We created "Dummy tasks", randomly selected code components not affected by any of the code smells considered in our study. This was done to limit the bias in the study, i.e., avoid that participants always indicate that the code contains a problem.
So, we first look for participants that accomplish tasks in a careless and/or biased way, analysing how they performed tasks when faced with dummy tasks. For instance, we intentionally created a dummy task that defines a god class, however the correspondent java code have just few line of codes. Thus, we expected that the participant would mark "I strongly disagree" or, at minimun, "I Slightly disagree". After data inspection, we noted that almost all of the participant not behave biased ou carelessly. Except for 1 participant, all of them marks some degree of disagreement when evaluated dummy tasks.

In [2]:
header = ["id","questionnaire_id","task_id","secondsToAnswer","isDt","answer_ptr_id","answer_csagreement","answer_description","answer_dtDescription"]
answer =  pd.read_csv('/mnt/d/Christiano/Documentos/dissertation/projeto/experiment/answer.csv', header=None, escapechar="\\", encoding="latin_1")
answer.columns=header

dummy_id = "13,14,15,16"
answer_dummy = answer.query("task_id in ({0})".format(dummy_id))

# o questionnaire id 25 será retirado
answer_dummy[answer_dummy.questionnaire_id==25].loc[:,'id':'answer_csagreement']

,id,questionnaire_id,task_id,secondsToAnswer,isDt,answer_ptr_id,answer_csagreement
85,187,25,14,83.19,0,187,-2
89,191,25,13,25.52,0,191,-2
93,195,25,16,89.50,1,195,2
95,197,25,15,143.85,1,197,2


In [17]:
# retira as atividades dummy
answer_no_dummy = answer[~answer.id.isin(answer_dummy.id)]

# retira alguns participantes para equalizar o quadrado latino
answer_no_dummy = answer_no_dummy[~answer_no_dummy.questionnaire_id.isin([50,68,55,25])]

# 12 participantes x 8 atividades = 96 rows
answer_no_dummy.head()

,id,questionnaire_id,task_id,secondsToAnswer,isDt,answer_ptr_id,answer_csagreement,answer_description,answer_dtDescription
0,102,16,3,119.14,1,102,2,The class is big and have too much code and va...,none
1,103,16,11,169.35,1,103,1,"It does have a long parameter list, could be a...","Helped me to see the number of statements, inp..."
2,104,16,7,250.75,1,104,1,The child class uses few methods from parent.,Helped me to identify why it was a refused beq...
3,105,16,4,235.66,1,105,-1,"It does too much in just one method, could be ...",helped me to identify where to search
8,110,16,8,42.01,0,110,2,The class is really long and implements too mu...,NaN


# Experiment results

In [88]:
import  statsmodels.stats.inter_rater as ir

def buildKappaTable(answer):    
    # calcula os agreements agrupando por atividade e agreement
    df_count_agreement = answer[['task_id','answer_csagreement']].groupby(['task_id','answer_csagreement']).size().reset_index(name='size')

    list_count_agreement = list()
    tasks = answer_no_dummy.task_id.unique()
    df_agreement_frame = pd.DataFrame([[-2,0],[-1,0],[1,0],[2,0]], columns=['answer_csagreement', 'size'])

    for task in tasks:
        df_tmp = df_count_agreement[df_count_agreement.task_id==task].iloc[:,1:3]
        df_merged = pd.merge_ordered(df_tmp, df_agreement_frame, fill_method='ffill', right_by="answer_csagreement", how="left")        
        list_count_agreement.append(df_merged['size'].tolist()) 
        
    # soma os agreements similares - agreement ou disagreement
    df_kappaTable = pd.DataFrame(list_count_agreement)
    df_disagree = df_kappaTable[0]+df_kappaTable[1]
    df_agree = df_kappaTable[2]+df_kappaTable[3]

    df_kappaTable = pd.concat([df_disagree, df_agree], axis=1)
    df_kappaTable.columns = ["Disagree", "Agree"]   
    df_kappaTable.index = tasks

    return df_kappaTable

def getFormattedTable(df_kappaTable):
    #relação de code smells e o código das atividades
    df_codeSmells = pd.DataFrame([[3,4,5,6,7,8,11,12],['gc','lm','rb','lm','rb','gc','lpl','lpl']])
    df_codeSmells = df_codeSmells.T
    df_codeSmells.columns = ['Task ID','CS']    
    df_kappaTable_formatted = pd.merge(df_kappaTable,df_codeSmells,left_index=True,right_on='Task ID')
    # reordena as colunas
    df_kappaTable_formatted = df_kappaTable_formatted[['Task ID','CS','Disagree','Agree']]
    #reordena as linhas por CS
    df_kappaTable_formatted = df_kappaTable_formatted.sort_values('CS')
    return df_kappaTable_formatted


answer_no_dummy_dt = answer_no_dummy[answer_no_dummy.isDt==1]
answer_no_dummy_noDt = answer_no_dummy[answer_no_dummy.isDt==0]

df_kappaTable_dt = buildKappaTable(answer_no_dummy_dt)
df_kappaTable_noDt = buildKappaTable(answer_no_dummy_noDt)

# cálculo do fleiss kappa
kappa_value_dt = ir.fleiss_kappa(df_kappaTable_dt, method='fleiss')
kappa_value_noDt = ir.fleiss_kappa(df_kappaTable_noDt, method='fleiss')

#tabelas formatadas com os agrees e disagrees
display(getFormattedTable(df_kappaTable_dt))
display(getFormattedTable(df_kappaTable_noDt))


,Task ID,CS,Disagree,Agree
0,3,gc,1,11
6,11,lpl,0,12
4,7,rb,5,7
1,4,lm,1,11
5,8,gc,2,10
7,12,lpl,5,7
3,6,lm,1,11
2,5,rb,6,6


,Task ID,CS,Disagree,Agree
0,3,gc,2,10
6,11,lpl,3,9
4,7,rb,4,8
1,4,lm,0,12
5,8,gc,1,11
7,12,lpl,6,6
3,6,lm,2,10
2,5,rb,5,7


In [86]:
#tabela com o kappa value de DT e noDT
df_kappa_values = pd.DataFrame([kappa_value_dt, kappa_value_noDt], columns=['Kappa agreement'], index=['DT','noDT'])

df_kappa_values

,Kappa agreement
DT,0.118961
noDT,0.059180


## Evaluation effort

In [0]:
answer_no_dummy_dt                  
    

labels = ['DT', 'No DT']

answerDtMinutes = answer_no_dummy_dt.secondsToAnswer
answerNoDtMinutes = answer_no_dummy_noDt.secondsToAnswer

all_data = [answerDtMinutes,answerNoDtMinutes]

plt.boxplot(all_data,
                         vert=True,  # vertical box alignment
                         patch_artist=True,  # fill with color
                         labels=labels)  # will be used to label x-ticks
#plt.show()

# df_table = pd.DataFrame([answerDtMinutes.tolist(),answerNoDtMinutes])
# df_table.T







In [0]:
import numpy as np

# Random test data
np.random.seed(19680801)
all_data = [np.random.normal(0, std, size=100) for std in range(1, 4)]
labels = ['x1', 'x2', 'x3']

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))

# rectangular box plot
bplot1 = axes[0].boxplot(all_data,
                         vert=True,  # vertical box alignment
                         patch_artist=True,  # fill with color
                         labels=labels)  # will be used to label x-ticks
axes[0].set_title('Rectangular box plot')

# notch shape box plot
bplot2 = axes[1].boxplot(all_data,
                         notch=True,  # notch shape
                         vert=True,  # vertical box alignment
                         patch_artist=True,  # fill with color
                         labels=labels)  # will be used to label x-ticks
axes[1].set_title('Notched box plot')

# fill with colors
colors = ['pink', 'lightblue', 'lightgreen']
for bplot in (bplot1, bplot2):
    for patch, color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)

# adding horizontal grid lines
for ax in axes:
    ax.yaxis.grid(True)
    ax.set_xlabel('Three separate samples')
    ax.set_ylabel('Observed values')

plt.show()

In [0]:
header = ["inviteId","codeRevision","codeSmellIdentification","degree","devExperience","javaExperience","objOrientedExperience","origin","yearsDevExperience"]
participants =  pd.read_csv('participant.csv', header=None)
participants.columns=header


#participants.to_csv('participant.csv', index=False)
        
#participants